# Songs of Scotland Prior to Burns Bot
This tool takes thirty tunes from Robert Chambers' Songs of Scotland Prior to Burns (SSPB), randomizes the measures, and returns new tunes to learn. SSPB can be found at https://archive.org/details/songsofscotlanpb00cham/page/450. Tunes were cross referenced with those in http://abcnotation.com/, saved in XML format, then parsed and modified largely with BeautifulSoup. All songs are in common time (4/4).

This corpus was great to work with as a beginner text miner because most songs are simple and in similar format.

# New songs are randomized in three possible ways: 
1. Randomly (all measures are mixed up and a 16 measure song is returned)
2. New by Note (all measures beginning with certain note are mixed up, and a 16 measure song is returned)
3. New by Key (all measures from songs of a certain key are mixed up, and a 16 measure song is returned)

In [61]:
from bs4 import BeautifulSoup
import glob
import random
import time

In [81]:
#grab files from directory
import os
path = "C:\\Users\\paige\\OneDrive\\Documents\\llcuCorpus"
os.chdir(path)
files = os.listdir()
files

['auld-lang-syne.xml',
 'bonny-katherine-ogie.xml',
 'last-time-i-came-oer-the-muir.xml',
 'logan-water.xml',
 'marys-dream.xml',
 'twas-within-a-mile-of-edinburgh-town.xml',
 'waukin-o-the-fauld.xml']

# *~Randomly ~*

In [82]:
penultimateMeasures = []

for filename in glob.glob(os.path.join(path, "*.xml")):#loop over, open, read and parse each xml file in directory
    with open(filename) as f:
        infile = open(filename,"r")
        contents = infile.read()
        soup = BeautifulSoup(contents,'xml')

        allMeasures = soup.find_all("measure")
        measures = allMeasures #copy and rename for organization
        for measure in measures: #loop over each measure found
            penultimateMeasures.append([measure]) #and add it to our list, penultimateMeasures (seen above)

#function to return new songs from all measures: RANDOMLY
def newSongsPen():
    random.shuffle(penultimateMeasures)
    return penultimateMeasures[0:16] #prints 16 measures of the randomization only

#newSongsPen()

Let it be known that I mix first and last measures in with the others despite slight differences in formatting because some of the corpus does not have keys at the start or barlines at the end of a song anyways. I did consider taking them out, but ultimately decided the option to change keys half way through songs would be cool.

In [72]:
#function returns new songs and adds xml encoding details back into the mix

def formatRandomSong():
    #create new soup to work with:
    newRandomSoup = BeautifulSoup(contents,'xml')
    
    #define date/time for below:
    now = time.strftime("%c")

    #change text inside elements to be in line with my SSPB bot:
    movementTitle = newRandomSoup.find("movement-title")
    movementTitle.string = "A New Song made from 'Songs of Scotland Prior to Burns' by Robert Chambers"
    encoder = newRandomSoup.encoder
    encoder.string = "The Songs of Scotland Prior to Burns Bot"
    encodingDate = newRandomSoup.find("encoding-date")
    encodingDate.string = now
    
    #decompose unecessary credit elements:
    newRandomSoup.credit.decompose()
    
    #inserts new, randomized measures, but first takes out all current measures
    #another way to do this would have been to take out measures, not part, but clear() didn't seem to work
    a_tag = newRandomSoup.find("part")
    new_tag = newRandomSoup.new_tag((newSongsPen()))
    a_tag.replace_with(new_tag)
    #add element part back in
    new_tag.wrap(newRandomSoup.new_tag("part"))

    return newRandomSoup

In [84]:
#calls and prints and random song from all measures!
formatRandomSong()

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE score-partwise PUBLIC "-//Recordare//DTD MusicXML 3.0 Partwise//EN" "http://www.musicxml.org/dtds/partwise.dtd">
<score-partwise>
<movement-title>A New Song made from 'Songs of Scotland Prior to Burns' by Robert Chambers</movement-title>
<identification>
<encoding>
<encoder>The Songs of Scotland Prior to Burns Bot</encoder>
<encoding-date>Mon Apr 15 15:44:55 2019</encoding-date>
</encoding>
</identification>

<part-list>
<score-part id="P1">
<part-name/>
</score-part>
</part-list>
<part><[[<measure number="5">
<note>
<pitch>
<step>F</step>
<octave>4</octave>
</pitch>
<duration>180</duration>
<voice>1</voice>
<type>quarter</type>
<dot/>
</note>
<note>
<pitch>
<step>G</step>
<octave>4</octave>
</pitch>
<duration>60</duration>
<voice>1</voice>
<type>eighth</type>
</note>
<note>
<pitch>
<step>A</step>
<octave>4</octave>
</pitch>
<duration>180</duration>
<voice>1</voice>
<type>quarter</type>
<dot/>
</note>
<note>
<pitch>
<step>G</step>
<octav

# *~New by Note~*

In [74]:
#gather and shuffle all measures beginning with the same note and print 16 of them
#does not work right now
D = []

for filename in glob.glob(os.path.join(path, "*.xml")):#loop over, open, read and parse each xml file in directory
    with open(filename) as f:
        infile = open(filename,"r")
        contents = infile.read()
        soup = BeautifulSoup(contents,'xml')
        allMeasures = soup.find_all("measure")
        for parentMeasure in allMeasures: #loop over each measure in allMeasures
            steps = soup.find("step") #and find the first instance of Step, which is the first note
            if step.get_text in steps == "D": #if the step is D
                parentMeasure = step.find_parent("measure") #find the parent measure
                D.append([parentMeasure]) #and add it to 
print(D)

def newSongsByNote():
    random.shuffle(D)
    return D[0:16]

[]


# *~New by Key~*

In [76]:
#to shuffle all songs with the same key and return 16 measures
penKeyMeasures = []

for filename in glob.glob(os.path.join(path, "*.xml")):#loop over, open, read and parse each xml file in directory
    with open(filename) as f:
        f = open(filename,"r")
        contents = f.read()
        soup = BeautifulSoup(contents,'xml')
        
        divisions = soup.find("divisions")
        fifths = soup.find("fifths")
        modes = soup.find("mode")
        #for key in soup (the following elements define the key of D major):
        if divisions.get_text() == "120":
            if fifths.get_text() == "2":
                if modes.get_text() == "major":
                    allKeyMeasures = soup.find_all("measure")
                    keyMeasures = allKeyMeasures[1:] #remove first measures with key signatures - key already known
                    for keyMeasure in keyMeasures: #loop over each measure
                            penKeyMeasures.append([keyMeasure]) #and add to penKeyMeasures list
                else:
                    print("That key is not found.") #if a key is not found, print as such

#function to return new songs from all measures: NEW BY KEY
def newSongsByKey():
    random.shuffle(penKeyMeasures)
    return penKeyMeasures[0:16]

#newSongsByKey()

In [77]:
def formatKeySong():
    #create new soup to work with:
    newKeySoup = BeautifulSoup(contents,'xml')
    
    #define date/time for below:
    now = time.strftime("%c")

    #change text inside elements to be in line with my SSPB bot:
    movementTitle = newKeySoup.find("movement-title")
    movementTitle.string = "A New Song made from 'Songs of Scotland Prior to Burns' by Robert Chambers"
    encoder = newKeySoup.encoder
    encoder.string = "The Songs of Scotland Prior to Burns Bot"
    encodingDate = newKeySoup.find("encoding-date")
    encodingDate.string = now
    
    #decompose unecessary credit elements:
    newKeySoup.credit.decompose()
    
    #inserts new, randomized measures, but first takes out all current measures
    #another way to do this would have been to take out measures, not part, but clear() didn't seem to work
    a_tag = newKeySoup.find("part")
    new_tag = newKeySoup.new_tag((newSongsByKey()))
    a_tag.replace_with(new_tag)
    #add element part back in
    new_tag.wrap(newKeySoup.new_tag("part"))

    return newKeySoup

In [85]:
#calls and prints and random song from all measures of songs in a certain key!
formatKeySong()

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE score-partwise PUBLIC "-//Recordare//DTD MusicXML 3.0 Partwise//EN" "http://www.musicxml.org/dtds/partwise.dtd">
<score-partwise>
<movement-title>A New Song made from 'Songs of Scotland Prior to Burns' by Robert Chambers</movement-title>
<identification>
<encoding>
<encoder>The Songs of Scotland Prior to Burns Bot</encoder>
<encoding-date>Mon Apr 15 15:45:14 2019</encoding-date>
</encoding>
</identification>

<part-list>
<score-part id="P1">
<part-name/>
</score-part>
</part-list>
<part><[[<measure number="17">
<note>
<pitch>
<step>F</step>
<alter>1</alter>
<octave>5</octave>
</pitch>
<duration>90</duration>
<voice>1</voice>
<type>eighth</type>
<dot/>
<beam number="1">begin</beam>
</note>
<note>
<pitch>
<step>E</step>
<octave>5</octave>
</pitch>
<duration>30</duration>
<voice>1</voice>
<type>16th</type>
<beam number="1">continue</beam>
</note>
<note>
<pitch>
<step>D</step>
<octave>5</octave>
</pitch>
<duration>90</duration>
<voice>1</voi

In [ ]:
#to do:
#16 measures by first note still doesn't work
#try to save file as xml/mxml and put into finale